### locations

In [0]:
CREATE STREAMING LIVE TABLE locations_bronze
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT * 
FROM cloud_files(
  "abfss://data@databricksacc2025.dfs.core.windows.net/openaq-locations-data/",
  "json",
  map(
    "inferSchema", "true",
    "multiLine", "true"
  )
);

In [0]:
CREATE STREAMING LIVE TABLE locations_silver
TBLPROPERTIES ("quality" = "silver")
AS
SELECT 
  CAST(id AS STRING) AS location_id,
  COALESCE(name, 'Unknown') AS name,
  COALESCE(locality, 'Unknown') AS locality,
  from_json(country, 'STRUCT<code: STRING, name: STRING>').code AS country_code,
  from_json(country, 'STRUCT<code: STRING, name: STRING>').name AS country_name,
  from_json(coordinates, 'STRUCT<latitude: DOUBLE, longitude: DOUBLE>').latitude AS latitude,
  from_json(coordinates, 'STRUCT<latitude: DOUBLE, longitude: DOUBLE>').longitude AS longitude
FROM STREAM(locations_bronze)

### measurements

In [0]:
CREATE STREAMING LIVE TABLE measurements_bronze
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT *
FROM cloud_files(
  "abfss://data@databricksacc2025.dfs.core.windows.net/openaq-measurements-data/",
  "json",
  map(
    "inferSchema", "true",
    "multiLine", "true"
  )
);

In [0]:
CREATE STREAMING LIVE TABLE measurements_silver
TBLPROPERTIES ("quality" = "silver")
AS
SELECT 
    CAST(location_id AS STRING) AS location_id,
    CAST(sensor_id AS STRING) AS sensor_id,
    from_json(parameter, 'STRUCT<id: STRING, name: STRING, units: STRING, displayName: STRING>').id AS parameter_id,
    from_json(parameter, 'STRUCT<id: STRING, name: STRING, units: STRING, displayName: STRING>').name AS parameter_name,
    from_json(parameter, 'STRUCT<id: STRING, name: STRING, units: STRING, displayName: STRING>').units AS parameter_units,
    from_json(parameter, 'STRUCT<id: STRING, name: STRING, units: STRING, displayName: STRING>').displayName AS parameter_displayName,
    ROUND(COALESCE(CAST(value AS DOUBLE), 0), 2) AS value,
    from_json(period, 'STRUCT<datetimeFrom: STRUCT<utc: STRING>, datetimeTo: STRUCT<utc: STRING>>').datetimeFrom.utc AS period_datetimeFrom_utc,
    from_json(period, 'STRUCT<datetimeFrom: STRUCT<utc: STRING>, datetimeTo: STRUCT<utc: STRING>>').datetimeTo.utc AS period_datetimeTo_utc,
    from_json(coverage, 'STRUCT<observedInterval: STRING, percentCoverage: STRING>').observedInterval AS coverage_observedInterval,
    from_json(coverage, 'STRUCT<observedInterval: STRING, percentCoverage: STRING>').percentCoverage AS coverage_percentCoverage
FROM STREAM(measurements_bronze)
WHERE location_id IS NOT NULL 
  AND sensor_id IS NOT NULL 
  AND value IS NOT NULL;

### weather

In [0]:
CREATE STREAMING LIVE TABLE weather_bronze
TBLPROPERTIES ("quality" = "bronze")
AS
WITH parsed AS (
    SELECT
        location_id,
        latitude,
        longitude,
        from_json(
          hourly, 
          "STRUCT<
             time:ARRAY<STRING>,
             temperature_2m:ARRAY<DOUBLE>,
             relative_humidity_2m:ARRAY<DOUBLE>,
             dew_point_2m:ARRAY<DOUBLE>,
             apparent_temperature:ARRAY<DOUBLE>,
             pressure_msl:ARRAY<DOUBLE>,
             surface_pressure:ARRAY<DOUBLE>,
             precipitation:ARRAY<DOUBLE>,
             rain:ARRAY<DOUBLE>,
             snowfall:ARRAY<DOUBLE>,
             cloud_cover:ARRAY<DOUBLE>,
             cloud_cover_low:ARRAY<DOUBLE>,
             cloud_cover_mid:ARRAY<DOUBLE>,
             cloud_cover_high:ARRAY<DOUBLE>,
             shortwave_radiation:ARRAY<DOUBLE>,
             direct_radiation:ARRAY<DOUBLE>,
             direct_normal_irradiance:ARRAY<DOUBLE>,
             diffuse_radiation:ARRAY<DOUBLE>,
             global_tilted_irradiance:ARRAY<DOUBLE>,
             sunshine_duration:ARRAY<DOUBLE>,
             wind_speed_10m:ARRAY<DOUBLE>,
             wind_speed_100m:ARRAY<DOUBLE>,
             wind_direction_10m:ARRAY<DOUBLE>,
             wind_direction_100m:ARRAY<DOUBLE>,
             wind_gusts_10m:ARRAY<DOUBLE>,
             et0_fao_evapotranspiration:ARRAY<DOUBLE>,
             weather_code:ARRAY<INT>,
             snow_depth:ARRAY<DOUBLE>,
             vapour_pressure_deficit:ARRAY<DOUBLE>,
             soil_temperature_0_to_7cm:ARRAY<DOUBLE>,
             soil_temperature_7_to_28cm:ARRAY<DOUBLE>,
             soil_temperature_28_to_100cm:ARRAY<DOUBLE>,
             soil_temperature_100_to_255cm:ARRAY<DOUBLE>,
             soil_moisture_0_to_7cm:ARRAY<DOUBLE>,
             soil_moisture_7_to_28cm:ARRAY<DOUBLE>,
             soil_moisture_28_to_100cm:ARRAY<DOUBLE>,
             soil_moisture_100_to_255cm:ARRAY<DOUBLE>
          >"
        ) AS hourly_struct
    FROM cloud_files(
      "abfss://data@databricksacc2025.dfs.core.windows.net/openmeteo-weather-data/",
      "json",
      MAP(
        "inferSchema", "true",
        "multiLine", "true"
      )
    )
),
flattened AS (
    SELECT
        location_id,
        latitude,
        longitude,
        time_array[pos] AS measurement_time,
        temperature_2m_array[pos] AS temperature_2m,
        relative_humidity_2m_array[pos] AS relative_humidity_2m,
        dew_point_2m_array[pos] AS dew_point_2m,
        apparent_temperature_array[pos] AS apparent_temperature,
        pressure_msl_array[pos] AS pressure_msl,
        surface_pressure_array[pos] AS surface_pressure,
        precipitation_array[pos] AS precipitation,
        rain_array[pos] AS rain,
        snowfall_array[pos] AS snowfall,
        cloud_cover_array[pos] AS cloud_cover,
        cloud_cover_low_array[pos] AS cloud_cover_low,
        cloud_cover_mid_array[pos] AS cloud_cover_mid,
        cloud_cover_high_array[pos] AS cloud_cover_high,
        shortwave_radiation_array[pos] AS shortwave_radiation,
        direct_radiation_array[pos] AS direct_radiation,
        direct_normal_irradiance_array[pos] AS direct_normal_irradiance,
        diffuse_radiation_array[pos] AS diffuse_radiation,
        global_tilted_irradiance_array[pos] AS global_tilted_irradiance,
        sunshine_duration_array[pos] AS sunshine_duration,
        wind_speed_10m_array[pos] AS wind_speed_10m,
        wind_speed_100m_array[pos] AS wind_speed_100m,
        wind_direction_10m_array[pos] AS wind_direction_10m,
        wind_direction_100m_array[pos] AS wind_direction_100m,
        wind_gusts_10m_array[pos] AS wind_gusts_10m,
        et0_fao_evapotranspiration_array[pos] AS et0_fao_evapotranspiration,
        weather_code_array[pos] AS weather_code,
        snow_depth_array[pos] AS snow_depth,
        vapour_pressure_deficit_array[pos] AS vapour_pressure_deficit,
        soil_temperature_0_to_7cm_array[pos] AS soil_temperature_0_to_7cm,
        soil_temperature_7_to_28cm_array[pos] AS soil_temperature_7_to_28cm,
        soil_temperature_28_to_100cm_array[pos] AS soil_temperature_28_to_100cm,
        soil_temperature_100_to_255cm_array[pos] AS soil_temperature_100_to_255cm,
        soil_moisture_0_to_7cm_array[pos] AS soil_moisture_0_to_7cm,
        soil_moisture_7_to_28cm_array[pos] AS soil_moisture_7_to_28cm,
        soil_moisture_28_to_100cm_array[pos] AS soil_moisture_28_to_100cm,
        soil_moisture_100_to_255cm_array[pos] AS soil_moisture_100_to_255cm

    FROM (
      SELECT
        location_id,
        latitude,
        longitude,
        hourly_struct.time                      AS time_array,
        hourly_struct.temperature_2m            AS temperature_2m_array,
        hourly_struct.relative_humidity_2m      AS relative_humidity_2m_array,
        hourly_struct.dew_point_2m              AS dew_point_2m_array,
        hourly_struct.apparent_temperature      AS apparent_temperature_array,
        hourly_struct.pressure_msl              AS pressure_msl_array,
        hourly_struct.surface_pressure          AS surface_pressure_array,
        hourly_struct.precipitation             AS precipitation_array,
        hourly_struct.rain                      AS rain_array,
        hourly_struct.snowfall                  AS snowfall_array,
        hourly_struct.cloud_cover               AS cloud_cover_array,
        hourly_struct.cloud_cover_low           AS cloud_cover_low_array,
        hourly_struct.cloud_cover_mid           AS cloud_cover_mid_array,
        hourly_struct.cloud_cover_high          AS cloud_cover_high_array,
        hourly_struct.shortwave_radiation       AS shortwave_radiation_array,
        hourly_struct.direct_radiation          AS direct_radiation_array,
        hourly_struct.direct_normal_irradiance  AS direct_normal_irradiance_array,
        hourly_struct.diffuse_radiation         AS diffuse_radiation_array,
        hourly_struct.global_tilted_irradiance  AS global_tilted_irradiance_array,
        hourly_struct.sunshine_duration         AS sunshine_duration_array,
        hourly_struct.wind_speed_10m            AS wind_speed_10m_array,
        hourly_struct.wind_speed_100m           AS wind_speed_100m_array,
        hourly_struct.wind_direction_10m        AS wind_direction_10m_array,
        hourly_struct.wind_direction_100m       AS wind_direction_100m_array,
        hourly_struct.wind_gusts_10m            AS wind_gusts_10m_array,
        hourly_struct.et0_fao_evapotranspiration AS et0_fao_evapotranspiration_array,
        hourly_struct.weather_code              AS weather_code_array,
        hourly_struct.snow_depth                AS snow_depth_array,
        hourly_struct.vapour_pressure_deficit   AS vapour_pressure_deficit_array,
        hourly_struct.soil_temperature_0_to_7cm AS soil_temperature_0_to_7cm_array,
        hourly_struct.soil_temperature_7_to_28cm AS soil_temperature_7_to_28cm_array,
        hourly_struct.soil_temperature_28_to_100cm AS soil_temperature_28_to_100cm_array,
        hourly_struct.soil_temperature_100_to_255cm AS soil_temperature_100_to_255cm_array,
        hourly_struct.soil_moisture_0_to_7cm    AS soil_moisture_0_to_7cm_array,
        hourly_struct.soil_moisture_7_to_28cm   AS soil_moisture_7_to_28cm_array,
        hourly_struct.soil_moisture_28_to_100cm AS soil_moisture_28_to_100cm_array,
        hourly_struct.soil_moisture_100_to_255cm AS soil_moisture_100_to_255cm_array
      FROM parsed
    )
    LATERAL VIEW POSEXPLODE(time_array) AS pos, t
)
SELECT *
FROM flattened;

In [0]:
CREATE STREAMING LIVE TABLE weather_silver
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
    CAST(location_id AS STRING) AS location_id,
    ROUND(COALESCE(latitude, 0), 6) AS latitude,
    ROUND(COALESCE(longitude, 0), 6) AS longitude,
    measurement_time,
    ROUND(COALESCE(temperature_2m, 0), 2) AS temperature_2m,
    ROUND(COALESCE(relative_humidity_2m, 0), 2) AS relative_humidity_2m,
    ROUND(COALESCE(precipitation, 0), 2) AS precipitation,
    ROUND(COALESCE(wind_speed_10m, 0), 2) AS wind_speed_10m,
    ROUND(COALESCE(wind_gusts_10m, 0), 2) AS wind_gusts_10m,
    COALESCE(weather_code, 0) AS weather_code  
FROM STREAM(LIVE.weather_bronze)
WHERE location_id IS NOT NULL 
  AND latitude IS NOT NULL 
  AND longitude IS NOT NULL;

### daily_meteorology_gold

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW daily_meteorology_gold
AS
SELECT
  loc.location_id AS loc_location_id,
  DATE(meas.period_datetimeFrom_utc) AS date,
  meas.parameter_name,  -- Dodano informację o rodzaju zanieczyszczenia
  loc.name,
  loc.locality,
  loc.country_code,
  loc.country_name,
  loc.latitude,
  loc.longitude,
  ROUND(COALESCE(AVG(meas.value), 0), 2) AS avg_measurement_value,
  ROUND(COALESCE(AVG(weather.temperature_2m), 0), 2) AS avg_temperature_2m,
  ROUND(COALESCE(AVG(weather.relative_humidity_2m), 0), 2) AS avg_relative_humidity_2m,
  ROUND(COALESCE(AVG(weather.precipitation), 0), 2) AS avg_precipitation,
  ROUND(COALESCE(AVG(weather.wind_speed_10m), 0), 2) AS avg_wind_speed_10m,
  ROUND(COALESCE(AVG(weather.wind_gusts_10m), 0), 2) AS avg_wind_gusts_10m,
  COUNT(*) AS record_count
FROM
  live.locations_silver loc
LEFT JOIN
  live.measurements_silver meas
ON
  loc.location_id = meas.location_id
LEFT JOIN
  live.weather_silver weather
ON
  loc.location_id = weather.location_id
  AND DATE(meas.period_datetimeFrom_utc) = DATE(weather.measurement_time)
WHERE
  loc.location_id IS NOT NULL
GROUP BY
  loc.location_id,
  DATE(meas.period_datetimeFrom_utc),
  meas.parameter_name,
  loc.name,
  loc.locality,
  loc.country_code,
  loc.country_name,
  loc.latitude,
  loc.longitude;